# Reading JWST ASDF-in-FITS with `astrowidgets`

This is a proof-of-concept using `astrowidgets` to read in JWST ASDF-in-FITS data. As it is using the dev versions of several different packages, I cannot really put it anywhere but in `dat_pyinthesky` repository for now.

Relevant specs used in testing:

* Linux (64-bit RHEL 7)
* Firefox 60.5.1esr (64-bit)
* Python 3.7.1
* `asdf` 2.3.1
* `astropy` 3.2.dev24004
* `astrowidgets` 0.1.0.dev111
* `ginga` 3.0.dev2166
* `gwcs` 0.10.0
* `ipython` 6.5.0
* `ipython_genutils` 0.2.0
* `jsonschema` 2.6.0
* `jupyter` 1.0.0
* `jupyter_client` 5.2.3
* `jupyter_console` 5.2.0
* `jupyter_core` 4.4.0
* `jwst` 0.13.1a0.dev67+g0d4a64da
* `notebook` 5.7.2
* `numpy` 1.15.4
* `requests` 2.20.0

**TODO:** Figure out how to hook up the custom IO handler with standalone Ginga application. The IO handler would ideally live in `stginga` in the future.

In [ ]:
import requests

from astropy.io import fits

from astrowidgets import ImageWidget

from ginga.AstroImage import AstroImage
from ginga.misc.log import get_logger
from ginga.util import wcsmod
from ginga.util.io_asdf import ASDFFileHandler

from jwst import datamodels

We need to ask Ginga to explicitly use its `astropy_ape14` WCS interface. This is unnecessary if every user sets it in their `~/.ginga/general.cfg` but that is not always guaranteed.

In [ ]:
wcsmod.use('astropy_ape14')

Now we define a generic function to download data file from Box to working directory locally. This function is not needed if you use an existing local copy of data file.

In [ ]:
def download_file(url):
    """Download into the current working directory the
    file from Box given the direct URL.
    
    Parameters
    ----------
    url : str
        URL to the file to be downloaded
        
    Returns
    -------
    download_filename : str
        Name of the downloaded file
    """
    response = requests.get(url, stream=True)
    if response.status_code != 200:
        raise RuntimeError("Wrong URL - {}".format(url))
    download_filename = response.headers['Content-Disposition'].split('"')[1]
    with open(download_filename, 'wb') as f:
        for chunk in response.iter_content(chunk_size=1024):
            if chunk:
                f.write(chunk)
    return download_filename

We use our `download_file()` function to download a simulated NIRCam image from STScI Box. This might take several seconds, depending on your bandwidth.

In [ ]:
url = 'https://stsci.box.com/shared/static/ofud7q1b8cogdm494iv9imvse8bk0uaj.fits'
filename = download_file(url)

In [ ]:
print(filename)

In [ ]:
with fits.open(filename) as pf:
    pf.info()

Next, we define a new `JWSTASDFHandler` class so Ginga knows how to read JWST ASDF-in-FITS file. In particular, this reads the data directly from the ASDF extension using `jwst.datamodels` and extracts the corresponding GWCS object.

**TODO:** https://github.com/ejeschke/ginga/issues/762

In [ ]:
class JWSTASDFHandler(ASDFFileHandler):
    factory_dict = {}

    @classmethod
    def register_type(cls, name, klass):
        cls.factory_dict[name.lower()] = klass
        
    def load_file(self, filespec, dstobj=None, **kwdargs):
        with datamodels.open(filespec) as dm:
            dstobj.setup_data(dm.data)
            dstobj.wcs.wcs = dm.meta.wcs  # Not dm.wcs!
            dstobj.wcs.coordsys = dm.meta.wcs.output_frame.name
        
        return dstobj

We register our new file handler with Ginga's `AstroImage` class. As Ginga is primarily an image viewer, we will not concern ourselves with spectrocopic data models.

In [ ]:
AstroImage.set_ioClass(JWSTASDFHandler)

Then, we customize our image widget by subclassing `ImageWidget` and adding a method to load the file.

**TODO:** We can even use `ImageWidget` if https://github.com/astropy/astrowidgets/pull/78 is merged, rendering this subclassing completely unnecessary.

In [ ]:
class JWSTImageWidget(ImageWidget):  
    def load_file(self, filename):
        image = AstroImage(logger=self.logger)
        image.load_file(filename)
        self._viewer.set_image(image)

We define a Ginga logger to go with our image widget.

In [ ]:
logger = get_logger('my viewer', log_stderr=True, log_file=None, level=30)

We create the widget instance. This would be the thing that you interface with for widget magic.

In [ ]:
w = JWSTImageWidget(logger=logger)

We load our JWST data file into our widget instance.

In [ ]:
w.load_file(filename)

This would display the widget. When you mouse over the pixels, you would see coordinates information (both pixels and sky) change. See https://astrowidgets.readthedocs.io/en/latest/ for documentation on `astrowidgets`.

In [ ]:
w